In [52]:
import numpy as np
import sys
sys.path.append("../src")
from data_processing import *
np.set_printoptions(suppress=True, precision=4, linewidth=120)

# **1. Khởi tạo và Đọc dữ liệu**
* **Xử lý sơ bộ:** Loại bỏ header và 2 cột cuối cùng (Naive Bayes Classifiers) không cần thiết.

In [53]:
file_path = "../data/raw/BankChurners.csv"

# 1. Đọc header để lấy tên cột
with open(file_path, "r") as f:
    header_line = f.readline().strip()
    raw_names = [c.strip('"') for c in header_line.split(",")]
    col_names = raw_names[:-2]

data_raw = np.genfromtxt(
    file_path, 
    delimiter=",", 
    dtype=str, 
    skip_header=1
)

data = data_raw[:, :-2]
data = np.char.strip(np.char.strip(data, '"'))

print("Kích thước dữ liệu:", data.shape)
print("Danh sách Features:", col_names)

Kích thước dữ liệu: (10127, 21)
Danh sách Features: ['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender', 'Dependent_count', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category', 'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']


# **2. Kiểm tra tính hợp lệ của dữ liệu (Data Validation)**
Trước khi xử lý, cần kiểm tra:
* **Data Types:** Đảm bảo các cột numerical thực sự là số
* **Range Validation:** Kiểm tra giá trị trong phạm vi hợp lệ
* **Duplicates:** Phát hiện dòng trùng lặp
* **Consistency:** Kiểm tra tính nhất quán của dữ liệu

In [54]:
print("KIỂM TRA TÍNH HỢP LỆ CỦA DỮ LIỆU")
 
# 1. Kiểm tra Duplicates
clientnum_idx = col_names.index("CLIENTNUM")
unique_clients = np.unique(data[:, clientnum_idx])
n_duplicates = len(data) - len(unique_clients)
print(f"\n1. Duplicates: {n_duplicates} dòng trùng lặp")

# 2. Định nghĩa các cột numerical sử dụng từ dataset
numerical_cols , _ = feature_typing(col_names , data)
numerical_cols = numerical_cols[1:]

# 3. Validation Rules
print("\n2. Validation Rules cho Numerical Columns:")
print(f"{'Column':<30} {'Min':<12} {'Max':<12} {'Valid Range':<20} {'Status'}")
print("-"*90)

validation_results = []
for col in numerical_cols:
    if col in col_names:
        idx = col_names.index(col)
        col_data = data[:, idx].astype(float)
        
        actual_min = np.min(col_data)
        actual_max = np.max(col_data)
        
        # Business logic constraints
        if col == "Avg_Utilization_Ratio":
            valid_min, valid_max = 0, 1
            constraint = "[0, 1]"
        elif col == "Customer_Age":
            valid_min, valid_max = 18, 120
            constraint = "[18, 120]"
        elif col in ["Months_Inactive_12_mon"]:
            valid_min, valid_max = 0, 12
            constraint = "[0, 12]"
        elif col in ["Credit_Limit", "Total_Revolving_Bal", "Total_Trans_Amt", "Total_Trans_Ct"]:
            valid_min, valid_max = 0, np.inf
            constraint = "[0, +inf)"
        else:
            valid_min, valid_max = -np.inf, np.inf
            constraint = "(-inf, +inf)"
        
        # Check violations
        violations = np.sum((col_data < valid_min) | (col_data > valid_max))
        status = "OK" if violations == 0 else f"{violations} vi phạm"
        
        validation_results.append({
            'column': col,
            'violations': violations,
            'actual_min': actual_min,
            'actual_max': actual_max
        })
        
        print(f"{col:<30} {actual_min:<12.2f} {actual_max:<12.2f} {constraint:<20} {status}")

# 4. Kiểm tra data type consistency
print("\n3. Data Type Consistency:")
for col in numerical_cols[:5]:  # Check first 5
    idx = col_names.index(col)
    try:
        _ = data[:, idx].astype(float)
        print(f"  {col:<30} Có thể convert sang float")
    except ValueError as e:
        print(f"  {col:<30} Lỗi: {e}")
 

KIỂM TRA TÍNH HỢP LỆ CỦA DỮ LIỆU

1. Duplicates: 0 dòng trùng lặp

2. Validation Rules cho Numerical Columns:
Column                         Min          Max          Valid Range          Status
------------------------------------------------------------------------------------------
Customer_Age                   26.00        73.00        [18, 120]            OK
Dependent_count                0.00         5.00         (-inf, +inf)         OK
Months_on_book                 13.00        56.00        (-inf, +inf)         OK
Total_Relationship_Count       1.00         6.00         (-inf, +inf)         OK
Months_Inactive_12_mon         0.00         6.00         [0, 12]              OK
Contacts_Count_12_mon          0.00         6.00         (-inf, +inf)         OK
Credit_Limit                   1438.30      34516.00     [0, +inf)            OK
Total_Revolving_Bal            0.00         2517.00      [0, +inf)            OK
Avg_Open_To_Buy                3.00         34516.00     (-inf, +i

# **3. Xử lý Missing Values**
Xác định và xử lý các giá trị bị thiếu hoặc giá trị rác (`Unknown`).
* **Chiến lược cho biến phân loại (Categorical):** Thay thế `Unknown` bằng **Mode** (giá trị xuất hiện nhiều nhất trong cột đó).
* **Chiến lược cho biến số (Numerical):** Nếu có `NaN`, thay thế bằng **Mean** hoặc **Median**.

In [55]:
missing_token = "Unknown"

# Xử lý Missing Values cho các cột Categorical
_ , categorical_cols_to_fix = feature_typing(col_names , data)
print("Xử lý Missing Values (Categorical)\n")

for name in categorical_cols_to_fix:
    if name in col_names:
        idx = col_names.index(name)
        col_data = data[:, idx]
        
        # Xác định vị trí missing
        mask_missing = (col_data == missing_token)
        count_missing = np.sum(mask_missing)
        
        if count_missing > 0:
            # Tính Mode trên dữ liệu sạch (không chứa Unknown)
            valid_data = col_data[~mask_missing]
            vals, counts = np.unique(valid_data, return_counts=True)
            mode_val = vals[np.argmax(counts)]
            
            # Thay thế Unknown bằng Mode
            data[:, idx] = np.where(mask_missing, mode_val, col_data)
            
            print(f"Cột {name:20s}: Đã thay thế {count_missing:4d} dòng '{missing_token}' => Mode: '{mode_val}'")
        else:
            print(f"Cột {name:20s}: Không có giá trị missing.")

idx_cl = col_names.index("Credit_Limit")
col_cl = data[:, idx_cl]

# Xử lý Missing Values cho cột Credit_Limit (Numeric)
try:
    col_float = col_cl.astype(float)
    if np.isnan(col_float).any():
        mean_val = np.nanmean(col_float)
        mask_nan = np.isnan(col_float)
        data[mask_nan, idx_cl] = str(mean_val)
        print("Đã xử lý NaN cho Credit_Limit")
except ValueError:
    pass

Xử lý Missing Values (Categorical)

Cột Attrition_Flag      : Không có giá trị missing.
Cột Gender              : Không có giá trị missing.
Cột Education_Level     : Đã thay thế 1519 dòng 'Unknown' => Mode: 'Graduate'
Cột Marital_Status      : Đã thay thế  749 dòng 'Unknown' => Mode: 'Married'
Cột Income_Category     : Đã thay thế 1112 dòng 'Unknown' => Mode: 'Less than $40K'
Cột Card_Category       : Không có giá trị missing.


# **4. Feature Engineering (Tạo đặc trưng mới)**
Tạo thêm các biến mới để hỗ trợ mô hình học máy tốt hơn:
1.  **Avg_Transaction_Value**: Giá trị trung bình mỗi giao dịch ($Total\_Trans\_Amt / Total\_Trans\_Ct$).
2.  **Utilization_Ratio**: Tỷ lệ sử dụng hạn mức tín dụng ($Total\_Revolving\_Bal / Credit\_Limit$).

Các phép tính số học được thực hiện cẩn thận để tránh lỗi chia cho 0 (ZeroDivisionError).

In [56]:
# Lấy index các cột cần dùng
idx_amt = col_names.index("Total_Trans_Amt")
idx_ct  = col_names.index("Total_Trans_Ct")
idx_bal = col_names.index("Total_Revolving_Bal")
idx_lim = col_names.index("Credit_Limit")

# Chuyển đổi dữ liệu sang dạng số (float) để tính toán
amt = data[:, idx_amt].astype(float)
ct  = data[:, idx_ct].astype(float)
bal = data[:, idx_bal].astype(float)
lim = data[:, idx_lim].astype(float)

# 1. Feature: Average Transaction Value
# Sử dụng np.maximum(ct, 1) để đảm bảo mẫu số tối thiểu là 1, tránh chia cho 0
avg_trans_val = amt / np.maximum(ct, 1)

# 2. Feature: Utilization Ratio
util_ratio = bal / np.maximum(lim, 1)

# Ghép các feature mới vào dataset
avg_trans_val_str = np.char.mod('%.4f', avg_trans_val)
util_ratio_str = np.char.mod('%.4f', util_ratio)

# Reshape để ghép cột
new_features = np.column_stack((avg_trans_val_str, util_ratio_str))
data = np.hstack((data, new_features))

# Cập nhật danh sách tên cột
col_names.extend(["Avg_Transaction_Value", "Utilization_Ratio"])

print("Kích thước dữ liệu sau khi thêm Features:", data.shape)
print("Features mới:", col_names[-2:])

Kích thước dữ liệu sau khi thêm Features: (10127, 23)
Features mới: ['Avg_Transaction_Value', 'Utilization_Ratio']


# **5. Feature Engineering - Tạo thêm features từ domain knowledge**
Dựa trên hiểu biết về business banking, tạo thêm các features có ý nghĩa:
1. **Transaction_Frequency:** Tần suất giao dịch trung bình (transactions/month)
2. **Inactive_Ratio:** Tỷ lệ thời gian không hoạt động
3. **Credit_Usage_Category:** Phân loại mức độ sử dụng tín dụng
4. **Customer_Lifetime_Value (CLV):** Ước lượng giá trị khách hàng

In [57]:

print("FEATURE ENGINEERING - TẠO THÊM FEATURES MỚI")

# Lấy các columns cần thiết
idx_months = col_names.index("Months_on_book")
idx_inactive = col_names.index("Months_Inactive_12_mon")
idx_trans_ct = col_names.index("Total_Trans_Ct")
idx_trans_amt = col_names.index("Total_Trans_Amt")
idx_rel_count = col_names.index("Total_Relationship_Count")

months_on_book = data[:, idx_months].astype(float)
months_inactive = data[:, idx_inactive].astype(float)
trans_ct = data[:, idx_trans_ct].astype(float)
trans_amt = data[:, idx_trans_amt].astype(float)
rel_count = data[:, idx_rel_count].astype(float)

# Feature 3: Transaction Frequency (transactions per month)
trans_frequency = trans_ct / np.maximum(months_on_book, 1)

# Feature 4: Inactive Ratio
inactive_ratio = months_inactive / 12.0  # Tỷ lệ inactive trong 12 tháng

# Feature 5: Customer Lifetime Value (CLV) estimate
# CLV = Avg Transaction Amount * Transaction Frequency * Months on Book * Relationship Count
clv = (trans_amt / np.maximum(trans_ct, 1)) * trans_frequency * months_on_book * rel_count

# Feature 6: Engagement Score (composite)
# Engagement = (1 - inactive_ratio) * transaction_frequency * relationship_count
engagement_score = (1 - inactive_ratio) * trans_frequency * rel_count

# Thêm vào dataset
new_features_2 = np.column_stack([
    np.char.mod('%.6f', trans_frequency),
    np.char.mod('%.6f', inactive_ratio),
    np.char.mod('%.6f', clv),
    np.char.mod('%.6f', engagement_score)
])

data = np.hstack((data, new_features_2))

new_feature_names = [
    "Transaction_Frequency",
    "Inactive_Ratio", 
    "Customer_Lifetime_Value",
    "Engagement_Score"
]

col_names.extend(new_feature_names)

print(f"\nĐã tạo {len(new_feature_names)} engineered features:")
for i, fname in enumerate(new_feature_names):
    print(f"   {i+1}. {fname}")

print(f"\nKích thước dữ liệu sau Feature Engineering: {data.shape}")
print(f"Tổng số features: {len(col_names)}")
 

FEATURE ENGINEERING - TẠO THÊM FEATURES MỚI

Đã tạo 4 engineered features:
   1. Transaction_Frequency
   2. Inactive_Ratio
   3. Customer_Lifetime_Value
   4. Engagement_Score

Kích thước dữ liệu sau Feature Engineering: (10127, 27)
Tổng số features: 27


# **6. Xử lý Outliers (Giá trị ngoại lai)**
Sử dụng phương pháp **Z-Score** để phát hiện và xử lý ngoại lai cho các biến số.
Công thức Z-score:
$$Z = \frac{x - \mu}{\sigma}$$

* **Ngưỡng:** $|Z| > 3$.
* **Phương pháp xử lý:** **Capping (Winsorizing)** - Thay thế giá trị vượt ngưỡng bằng giá trị biên (Mean $\pm$ 3*Std) thay vì xóa bỏ dòng dữ liệu.

In [58]:
def cap_outliers_zscore(values):
    
    values = values.astype(float)
    mean = np.mean(values)
    std = np.std(values)
    
    if std == 0: return values, 0
    
    threshold = 3
    upper_bound = mean + threshold * std
    lower_bound = mean - threshold * std
    
    # Đếm outliers
    n_outliers = np.sum((values > upper_bound) | (values < lower_bound))
    
    # Capping (Kẹp giá trị trong khoảng)
    values_clipped = np.clip(values, lower_bound, upper_bound)
    
    return values_clipped, n_outliers

# Áp dụng cho một số biến số quan trọng
numeric_cols_to_clean = ["Credit_Limit", "Total_Trans_Amt", "Avg_Transaction_Value"]

print("Xử lý Outliers")
for name in numeric_cols_to_clean:
    idx = col_names.index(name)
    col_vals = data[:, idx]
    
    # Xử lý
    cleaned_vals, n_out = cap_outliers_zscore(col_vals)
    
    # Cập nhật lại vào data (ép về dạng string)
    data[:, idx] = np.char.mod('%.4f', cleaned_vals)
    
    print(f"{name:25s}: Đã xử lý {n_out:4d} outliers (Capping)")

Xử lý Outliers
Credit_Limit             : Đã xử lý    0 outliers (Capping)
Total_Trans_Amt          : Đã xử lý  391 outliers (Capping)
Avg_Transaction_Value    : Đã xử lý  244 outliers (Capping)


# **7. Phương pháp IQR (Interquartile Range) để phát hiện Outliers**
Phương pháp IQR là một cách robust hơn Z-score, không bị ảnh hưởng bởi outliers cực đoan.

**Công thức:**
* $IQR = Q_3 - Q_1$
* **Lower Bound:** $Q_1 - 1.5 \times IQR$
* **Upper Bound:** $Q_3 + 1.5 \times IQR$

**So sánh:** IQR vs Z-Score để quyết định có nên loại bỏ outliers không.

In [59]:
def detect_outliers_iqr(values, multiplier=1.5):

    values = values.astype(float)
    q1 = np.percentile(values, 25)
    q3 = np.percentile(values, 75)
    iqr = q3 - q1
    
    lower_bound = q1 - multiplier * iqr
    upper_bound = q3 + multiplier * iqr
    
    outliers = (values < lower_bound) | (values > upper_bound)
    n_outliers = np.sum(outliers)
    
    return {
        'q1': q1,
        'q3': q3,
        'iqr': iqr,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound,
        'n_outliers': n_outliers,
        'outlier_pct': (n_outliers / len(values)) * 100
    }


# So sánh Z-Score vs IQR 
print("SO SÁNH PHƯƠNG PHÁP PHÁT HIỆN OUTLIERS: Z-SCORE vs IQR")
 

test_col = "Total_Trans_Amt"
idx_test = col_names.index(test_col)
test_values = data[:, idx_test].astype(float)

# Phương pháp 1: Z-Score
mean_val = np.mean(test_values)
std_val = np.std(test_values)
z_scores = np.abs((test_values - mean_val) / std_val)
outliers_zscore = z_scores > 3
n_outliers_zscore = np.sum(outliers_zscore)

# Phương pháp 2: IQR
iqr_result = detect_outliers_iqr(test_values, multiplier=1.5)

print(f"\nCột phân tích: {test_col}")
print(f"Tổng số samples: {len(test_values)}")
print("\n1. Z-Score Method (threshold=3):")
print(f"   Outliers detected: {n_outliers_zscore} ({n_outliers_zscore/len(test_values)*100:.2f}%)")
print(f"   Bounds: Mean ± 3*Std = [{mean_val - 3*std_val:.2f}, {mean_val + 3*std_val:.2f}]")

print("\n2. IQR Method (multiplier=1.5):")
print(f"   Q1 = {iqr_result['q1']:.2f}, Q3 = {iqr_result['q3']:.2f}, IQR = {iqr_result['iqr']:.2f}")
print(f"   Outliers detected: {iqr_result['n_outliers']} ({iqr_result['outlier_pct']:.2f}%)")
print(f"   Bounds: [{iqr_result['lower_bound']:.2f}, {iqr_result['upper_bound']:.2f}]")

print("\nPhân tích và Quyết định:")
if iqr_result['outlier_pct'] < 5:
    print("   Tỷ lệ outliers < 5% -> CÓ THỂ XÓA BỎ nếu cần")
    print("   NHƯNG: Nên dùng CAPPING/WINSORIZING để giữ lại thông tin")
elif iqr_result['outlier_pct'] < 10:
    print("   Tỷ lệ outliers 5-10% -> NÊN CAPPING thay vì xóa")
else:
    print("   Tỷ lệ outliers > 10% -> KHÔNG NÊN XÓA, dùng Robust methods")

print("\nKết luận:")
print("   Trong trường hợp này, ta sử dụng CAPPING (đã thực hiện ở bước 4)")
print("   Lý do: Giữ lại toàn bộ samples, tránh mất thông tin quan trọng")
 

SO SÁNH PHƯƠNG PHÁP PHÁT HIỆN OUTLIERS: Z-SCORE vs IQR

Cột phân tích: Total_Trans_Amt
Tổng số samples: 10127

1. Z-Score Method (threshold=3):
   Outliers detected: 504 (4.98%)
   Bounds: Mean ± 3*Std = [-5477.23, 14215.23]

2. IQR Method (multiplier=1.5):
   Q1 = 2155.50, Q3 = 4741.00, IQR = 2585.50
   Outliers detected: 896 (8.85%)
   Bounds: [-1722.75, 8619.25]

Phân tích và Quyết định:
   Tỷ lệ outliers 5-10% -> NÊN CAPPING thay vì xóa

Kết luận:
   Trong trường hợp này, ta sử dụng CAPPING (đã thực hiện ở bước 4)
   Lý do: Giữ lại toàn bộ samples, tránh mất thông tin quan trọng


# **8. Chuẩn hóa và Điều chuẩn dữ liệu**
Thực hiện các kỹ thuật scaling thủ công:
1.  **Min-Max Scaling:** Đưa dữ liệu về đoạn $[0, 1]$.
    $$X_{norm} = \frac{X - min}{max - min}$$
2.  **Standardization (Z-score Scaling):** Đưa dữ liệu về phân phối chuẩn ($\mu=0, \sigma=1$).
    $$X_{std} = \frac{X - \mu}{\sigma}$$
3.  **Log Transformation:** Giảm độ lệch (skewness).
    $$X_{log} = \ln(X + 1)$$

In [60]:
idx_target = col_names.index("Total_Trans_Amt")
raw_feature = data[:, idx_target].astype(float)

feat_minmax = min_max_scale(raw_feature)
feat_std = standard_scale(raw_feature)
feat_log = log_transform(raw_feature)

print(f"Biến đổi đặc trưng: {col_names[idx_target]}")
print(f"{'Index':<5} | {'Original':<10} | {'MinMax':<10} | {'Standard':<10} | {'Log':<10}")
print("-" * 55)
for i in range(5):
    print(f"{i:<5} | {raw_feature[i]:<10.2f} | {feat_minmax[i]:<10.4f} | {feat_std[i]:<10.4f} | {feat_log[i]:<10.4f}")

print(f"\nKiểm tra Standardization -> Mean: {np.mean(feat_std):.5f} (approx 0), Std: {np.std(feat_std):.5f} (approx 1)")

Biến đổi đặc trưng: Total_Trans_Amt
Index | Original   | MinMax     | Standard   | Log       
-------------------------------------------------------
0     | 1144.00    | 0.0450     | -0.9826    | 7.0432    
1     | 1291.00    | 0.0554     | -0.9378    | 7.1639    
2     | 1887.00    | 0.0978     | -0.7562    | 7.5433    
3     | 1171.00    | 0.0469     | -0.9744    | 7.0665    
4     | 816.00     | 0.0217     | -1.0825    | 6.7056    

Kiểm tra Standardization -> Mean: -0.00000 (approx 0), Std: 1.00000 (approx 1)


# **9. Áp dụng Scaling cho toàn bộ Numerical Features**
Chiến lược scaling dựa trên distribution của từng feature:
* **Standardization (Z-score):** Cho features có phân phối gần Gaussian
* **Min-Max:** Cho features có distribution uniform hoặc bounded
* **Log Transform:** Cho features có skewness cao (right-skewed)

In [61]:
def calculate_skewness(arr):
    arr = arr.astype(float)
    mean = np.mean(arr)
    std = np.std(arr)
    if std == 0: return 0
    n = len(arr)
    return (n / ((n-1) * (n-2))) * np.sum(((arr - mean) / std) ** 3)

 
print("ÁP DỤNG SCALING CHO NUMERICAL FEATURES")
 

# Tạo dict để lưu scaled data
scaled_data = data.copy()

# Xác định numerical columns để scale
numerical_cols_to_scale , _ = feature_typing(col_names , data)
numerical_cols_to_scale = numerical_cols_to_scale[1:]

print(f"\nPhân tích Distribution và chọn Scaling Method:")
print(f"{'Column':<30} {'Skewness':<12} {'Method':<20}")
print("-"*65)

scaling_methods = {}

for col in numerical_cols_to_scale:
    if col in col_names:
        idx = col_names.index(col)
        col_data = data[:, idx].astype(float)
        
        # Tính skewness
        skew = calculate_skewness(col_data)
        
        # Quyết định method dựa trên skewness
        if abs(skew) < 0.5:
            method = "Standardization"
            scaled = standard_scale(col_data)
        elif abs(skew) < 1.0:
            method = "Min-Max"
            scaled = min_max_scale(col_data)
        else:
            method = "Log + Standardization"
            scaled = standard_scale(log_transform(col_data))
        
        # Update scaled data
        scaled_data[:, idx] = np.char.mod('%.6f', scaled)
        scaling_methods[col] = method
        
        print(f"{col:<30} {skew:<12.3f} {method:<20}")
 

ÁP DỤNG SCALING CHO NUMERICAL FEATURES

Phân tích Distribution và chọn Scaling Method:
Column                         Skewness     Method              
-----------------------------------------------------------------

Phân tích Distribution và chọn Scaling Method:
Column                         Skewness     Method              
-----------------------------------------------------------------
Customer_Age                   -0.034       Standardization     
Customer_Age                   -0.034       Standardization     
Dependent_count                -0.021       Standardization     
Months_on_book                 -0.107       Standardization     
Total_Relationship_Count       -0.162       Standardization     
Dependent_count                -0.021       Standardization     
Months_on_book                 -0.107       Standardization     
Total_Relationship_Count       -0.162       Standardization     
Months_Inactive_12_mon         0.633        Min-Max             
Contacts_Count_12_

# **10. Encoding Categorical Variables - One-Hot Encoding**
Chuyển đổi categorical features thành numerical format bằng One-Hot Encoding thủ công.

**Ví dụ:** `Gender` (M, F) → `Gender_M` (0/1), `Gender_F` (0/1)

In [62]:
print("ONE-HOT ENCODING CHO CATEGORICAL VARIABLES")
 

# Categorical columns cần encode
_ , categorical_cols = feature_typing(col_names , data)
categorical_cols = [col for col in categorical_cols if col != "Attrition_Flag"]

encoded_features = []
encoded_col_names = []

for col in categorical_cols:
    if col in col_names:
        idx = col_names.index(col)
        col_data = scaled_data[:, idx]
        
        # One-hot encoding
        encoded_matrix, categories = one_hot_encode_manual(col_data)
        
        # Tạo tên cho các encoded columns
        new_names = [f"{col}_{cat}" for cat in categories]
        
        encoded_features.append(encoded_matrix)
        encoded_col_names.extend(new_names)
        
        print(f"\n{col}:")
        print(f"  Categories: {categories}")
        print(f"  Encoded shape: {encoded_matrix.shape}")
        print(f"  New column names: {new_names}")

# Combine tất cả encoded features
all_encoded = np.hstack(encoded_features)

print(f"\n{'='*80}")
print(f"Encode {len(categorical_cols)} categorical columns")
print(f"Tổng số encoded features: {all_encoded.shape[1]}")
print(f"{'='*80}")

ONE-HOT ENCODING CHO CATEGORICAL VARIABLES



Gender:
  Categories: ['F' 'M']
  Encoded shape: (10127, 2)
  New column names: ['Gender_F', 'Gender_M']

Education_Level:
  Categories: ['College' 'Doctorate' 'Graduate' 'High School' 'Post-Graduate' 'Uneducated']
  Encoded shape: (10127, 6)
  New column names: ['Education_Level_College', 'Education_Level_Doctorate', 'Education_Level_Graduate', 'Education_Level_High School', 'Education_Level_Post-Graduate', 'Education_Level_Uneducated']

Marital_Status:
  Categories: ['Divorced' 'Married' 'Single']
  Encoded shape: (10127, 3)
  New column names: ['Marital_Status_Divorced', 'Marital_Status_Married', 'Marital_Status_Single']

Income_Category:
  Categories: ['$120K +' '$40K - $60K' '$60K - $80K' '$80K - $120K' 'Less than $40K']
  Encoded shape: (10127, 5)
  New column names: ['Income_Category_$120K +', 'Income_Category_$40K - $60K', 'Income_Category_$60K - $80K', 'Income_Category_$80K - $120K', 'Income_Category_Less than $40K']

Card_Category:
  Categories: ['Blue' 'Gold' 'Platinum' 'Si

# **11. Kiểm định Giả thiết Thống kê (Hypothesis Testing)**
Sử dụng **Z-test** (cho mẫu lớn) để kiểm định sự khác biệt giữa hai nhóm khách hàng.

**Bài toán:** So sánh Hạn mức tín dụng trung bình (`Credit_Limit`) giữa Khách hàng Nam và Nữ.

* **Giả thiết $H_0$:** $\mu_{Male} = \mu_{Female}$ (Không có sự khác biệt).
* **Giả thiết $H_1$:** $\mu_{Male} \neq \mu_{Female}$ (Có sự khác biệt).
* **Mức ý nghĩa $\alpha$:** 0.05.

Công thức Z-statistic:
$$Z = \frac{\bar{X}_1 - \bar{X}_2}{\sqrt{\frac{\sigma_1^2}{n_1} + \frac{\sigma_2^2}{n_2}}}$$

In [63]:
# 1. Tách nhóm dữ liệu
idx_gen = col_names.index("Gender")
idx_lim = col_names.index("Credit_Limit")

# Lấy dữ liệu Credit Limit (đã clean ở bước Outlier) và ép kiểu float
group_m = data[data[:, idx_gen] == "M", idx_lim].astype(float)
group_f = data[data[:, idx_gen] == "F", idx_lim].astype(float)

# 2. Tính toán các chỉ số thống kê
n1, n2 = len(group_m), len(group_f)
mean1, mean2 = np.mean(group_m), np.mean(group_f)
# ddof=1 để tính độ lệch chuẩn mẫu (sample standard deviation)
std1, std2 = np.std(group_m, ddof=1), np.std(group_f, ddof=1)

print(f"Nam (M): n={n1}, mean={mean1:.2f}, std={std1:.2f}")
print(f"Nữ  (F): n={n2}, mean={mean2:.2f}, std={std2:.2f}")

# 3. Tính Z-score
numerator = mean1 - mean2
denominator = np.sqrt((std1**2 / n1) + (std2**2 / n2))
z_stat = numerator / denominator

print(f"\nZ-statistic: {z_stat:.4f}")

# 4. Kết luận
# Với alpha = 0.05 (2 phía), ngưỡng tới hạn (Critical Value) là 1.96
critical_val = 1.96

if abs(z_stat) > critical_val:
    print(f"Kết quả: |{z_stat:.2f}| > {critical_val} => Bác bỏ H0.")
    print("Kết luận: Có sự khác biệt có ý nghĩa thống kê về hạn mức tín dụng giữa Nam và Nữ.")
else:
    print(f"Kết quả: |{z_stat:.2f}| <= {critical_val} => Không đủ cơ sở bác bỏ H0.")

Nam (M): n=4769, mean=12685.67, std=10647.94
Nữ  (F): n=5358, mean=5023.85, std=5251.88

Z-statistic: 45.0524
Kết quả: |45.05| > 1.96 => Bác bỏ H0.
Kết luận: Có sự khác biệt có ý nghĩa thống kê về hạn mức tín dụng giữa Nam và Nữ.


# **12. Kiểm định Giả thiết - Chi-Square Test**
**Bài toán:** Kiểm định mối quan hệ giữa `Card_Category` và `Attrition_Flag`.

* **Giả thiết $H_0$:** Card Category và Attrition độc lập với nhau.
* **Giả thiết $H_1$:** Card Category và Attrition có liên hệ với nhau.
* **Mức ý nghĩa $\alpha$:** 0.05.

**Công thức Chi-square:**
$$\chi^2 = \sum \frac{(O_{ij} - E_{ij})^2}{E_{ij}}$$

với $E_{ij} = \frac{\text{Row}_i \times \text{Col}_j}{N}$

In [64]:
print("CHI-SQUARE TEST: CARD CATEGORY vs ATTRITION")
 

# 1. Tạo Contingency Table
idx_card = col_names.index("Card_Category")
idx_attr = col_names.index("Attrition_Flag")

card_categories = np.unique(data[:, idx_card])
attrition_statuses = np.unique(data[:, idx_attr])

# Build contingency table
contingency_table = []
for card in card_categories:
    row = []
    for attr in attrition_statuses:
        count = np.sum((data[:, idx_card] == card) & (data[:, idx_attr] == attr))
        row.append(count)
    contingency_table.append(row)

contingency_table = np.array(contingency_table)

print("\nContingency Table:")
print(f"{'Card Category':<15}", end="")
for attr in attrition_statuses:
    print(f"{attr:<20}", end="")
print("Total")
print("-"*70)

for i, card in enumerate(card_categories):
    print(f"{card:<15}", end="")
    row_sum = 0
    for j in range(len(attrition_statuses)):
        print(f"{contingency_table[i, j]:<20}", end="")
        row_sum += contingency_table[i, j]
    print(row_sum)

# 2. Chi-square test
chi2, p_value, df, expected = chi_square_test_manual(contingency_table)

print(f"\nKiểm định giả thiết:")
print(f"   H0: Card Category và Attrition độc lập")
print(f"   H1: Card Category và Attrition có liên hệ")
print(f"   Significance level: α = 0.05")
print(f"\nKết quả:")
print(f"   Chi-square statistic: {chi2:.4f}")
print(f"   Degrees of freedom: {df}")
print(f"   P-value (approx): {p_value:.6f}")

if p_value < 0.05:
    print(f"\nKết luận: BÁC BỎ H0 (p={p_value:.6f} < 0.05)")
    print("   → Card Category và Attrition CÓ LIÊN HỆ có ý nghĩa thống kê")
else:
    print(f"\nKết luận: KHÔNG ĐỦ CƠ SỞ BÁC BỎ H0 (p={p_value:.6f} >= 0.05)")
    print("   → Chưa có bằng chứng cho thấy có liên hệ")

 

CHI-SQUARE TEST: CARD CATEGORY vs ATTRITION

Contingency Table:
Card Category  Attrited Customer   Existing Customer   Total
----------------------------------------------------------------------
Blue           1519                7917                9436
Gold           21                  95                  116
Platinum       5                   15                  20
Silver         82                  473                 555

Kiểm định giả thiết:
   H0: Card Category và Attrition độc lập
   H1: Card Category và Attrition có liên hệ
   Significance level: α = 0.05

Kết quả:
   Chi-square statistic: 2.2342
   Degrees of freedom: 3
   P-value (approx): 0.525301

Kết luận: KHÔNG ĐỦ CƠ SỞ BÁC BỎ H0 (p=0.525301 >= 0.05)
   → Chưa có bằng chứng cho thấy có liên hệ


# **13. Kiểm định Giả thiết - Independent T-Test**
**Bài toán:** So sánh Transaction Amount trung bình giữa Existing vs Attrited customers.

* **Giả thiết $H_0$:** $\mu_{Existing} = \mu_{Attrited}$.
* **Giả thiết $H_1$:** $\mu_{Existing} \neq \mu_{Attrited}$.
* **Mức ý nghĩa $\alpha$:** 0.05.

**Công thức t-statistic:**
$$t = \frac{\bar{X}_1 - \bar{X}_2}{\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}}$$

In [65]:
print("INDEPENDENT T-TEST: TRANSACTION AMOUNT - EXISTING vs ATTRITED")
 
# 1. Phân nhóm dữ liệu
idx_attr = col_names.index("Attrition_Flag")
idx_trans_amt = col_names.index("Total_Trans_Amt")

existing_mask = (data[:, idx_attr] == "Existing Customer")
attrited_mask = (data[:, idx_attr] == "Attrited Customer")

trans_existing = data[existing_mask, idx_trans_amt].astype(float)
trans_attrited = data[attrited_mask, idx_trans_amt].astype(float)

# 2. Thống kê mô tả
print(f"\nExisting Customers:")
print(f"  n = {len(trans_existing)}")
print(f"  Mean = {np.mean(trans_existing):.2f}")
print(f"  Std = {np.std(trans_existing, ddof=1):.2f}")

print(f"\nAttrited Customers:")
print(f"  n = {len(trans_attrited)}")
print(f"  Mean = {np.mean(trans_attrited):.2f}")
print(f"  Std = {np.std(trans_attrited, ddof=1):.2f}")

# 3. T-test
t_stat, p_value, df = t_test_independent_manual(trans_existing, trans_attrited)

print(f"\nKiểm định giả thiết:")
print(f"   H0: μ_Existing = μ_Attrited")
print(f"   H1: μ_Existing ≠ μ_Attrited")
print(f"   Significance level: α = 0.05")
print(f"\nKết quả:")
print(f"   t-statistic: {t_stat:.4f}")
print(f"   Degrees of freedom: {df:.2f}")
print(f"   P-value (two-tailed): {p_value:.10f}")

# Critical value cho α=0.05, two-tailed (approx 1.96 for large df)
if p_value < 0.05:
    print(f"\nKết luận: BÁC BỎ H0 (p={p_value:.10f} < 0.05)")
    print("   → Có sự khác biệt CÓ Ý NGHĨA THỐNG KÊ về Transaction Amount")
    diff = np.mean(trans_existing) - np.mean(trans_attrited)
    print(f"   → Existing customers có trung bình cao hơn {diff:.2f}")
else:
    print(f"\nKết luận: KHÔNG ĐỦ CƠ SỞ BÁC BỎ H0 (p={p_value:.6f} >= 0.05)")

 

INDEPENDENT T-TEST: TRANSACTION AMOUNT - EXISTING vs ATTRITED

Existing Customers:
  n = 8500
  Mean = 4612.85
  Std = 3383.19

Attrited Customers:
  n = 1627
  Mean = 3095.03
  Std = 2308.23

Kiểm định giả thiết:
   H0: μ_Existing = μ_Attrited
   H1: μ_Existing ≠ μ_Attrited
   Significance level: α = 0.05

Kết quả:
   t-statistic: 22.3276
   Degrees of freedom: 3136.72
   P-value (two-tailed): 0.0000000000

Kết luận: BÁC BỎ H0 (p=0.0000000000 < 0.05)
   → Có sự khác biệt CÓ Ý NGHĨA THỐNG KÊ về Transaction Amount
   → Existing customers có trung bình cao hơn 1517.83


# **14. Tạo Final Preprocessed Dataset và Lưu File**
Kết hợp tất cả:
1. Numerical features (đã scaled)
2. Engineered features
3. Encoded categorical features
4. Target variable

Sau đó lưu vào file `.npy` để sử dụng cho modeling.

In [66]:
print("TẠO FINAL PREPROCESSED DATASET")
 

# 1. Lấy numerical features (đã scaled)
numerical_indices = [col_names.index(col) for col in numerical_cols_to_scale if col in col_names]
X_numerical = scaled_data[:, numerical_indices].astype(float)

print(f"\n1. Numerical features (scaled): {X_numerical.shape}")

# 2. Lấy encoded categorical features
X_categorical = all_encoded

print(f"2. Categorical features (encoded): {X_categorical.shape}")

# 3. Combine tất cả features
X_final = np.hstack([X_numerical, X_categorical])

# 4. Target variable
idx_target = col_names.index("Attrition_Flag")
y = np.array([1 if val == "Attrited Customer" else 0 for val in data[:, idx_target]])

print(f"3. Combined features: {X_final.shape}")
print(f"4. Target variable: {y.shape}")
print(f"   - Class 0 (Existing): {np.sum(y==0)} samples")
print(f"   - Class 1 (Attrited): {np.sum(y==1)} samples")

# 5. Lưu vào file
import os

output_dir = "../data/processed"
os.makedirs(output_dir, exist_ok=True)

# Lưu features và target
np.save(os.path.join(output_dir, "X_preprocessed.npy"), X_final)
np.save(os.path.join(output_dir, "y_target.npy"), y)

# Lưu feature names
all_final_feature_names = [col for col in numerical_cols_to_scale if col in col_names] + encoded_col_names

with open(os.path.join(output_dir, "feature_names.txt"), "w") as f:
    for fname in all_final_feature_names:
        f.write(fname + "\n")

# Lưu metadata
with open(os.path.join(output_dir, "preprocessing_metadata.txt"), "w") as f:
    f.write("PREPROCESSING METADATA\n")
    f.write("="*80 + "\n\n")
    f.write(f"Dataset shape: {X_final.shape}\n")
    f.write(f"Number of samples: {X_final.shape[0]}\n")
    f.write(f"Number of features: {X_final.shape[1]}\n")
    f.write(f"  - Numerical (scaled): {X_numerical.shape[1]}\n")
    f.write(f"  - Categorical (encoded): {X_categorical.shape[1]}\n\n")
    f.write(f"Target distribution:\n")
    f.write(f"  - Class 0 (Existing): {np.sum(y==0)} ({np.sum(y==0)/len(y)*100:.2f}%)\n")
    f.write(f"  - Class 1 (Attrited): {np.sum(y==1)} ({np.sum(y==1)/len(y)*100:.2f}%)\n\n")
    f.write("Scaling methods applied:\n")
    for col, method in scaling_methods.items():
        f.write(f"  - {col}: {method}\n")

print(f"\nĐã lưu preprocessed data vào: {output_dir}/")
print(f"   - X_preprocessed.npy")
print(f"   - y_target.npy")
print(f"   - feature_names.txt")
print(f"   - preprocessing_metadata.txt")
 

TẠO FINAL PREPROCESSED DATASET

1. Numerical features (scaled): (10127, 20)
2. Categorical features (encoded): (10127, 20)
3. Combined features: (10127, 40)
4. Target variable: (10127,)
   - Class 0 (Existing): 8500 samples
   - Class 1 (Attrited): 1627 samples

Đã lưu preprocessed data vào: ../data/processed/
   - X_preprocessed.npy
   - y_target.npy
   - feature_names.txt
   - preprocessing_metadata.txt

1. Numerical features (scaled): (10127, 20)
2. Categorical features (encoded): (10127, 20)
3. Combined features: (10127, 40)
4. Target variable: (10127,)
   - Class 0 (Existing): 8500 samples
   - Class 1 (Attrited): 1627 samples

Đã lưu preprocessed data vào: ../data/processed/
   - X_preprocessed.npy
   - y_target.npy
   - feature_names.txt
   - preprocessing_metadata.txt
